In [64]:
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA

# from openai import OpenAI
# import chromadb

# Loading the environment which contains the OPENAI_API key
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import ifcopenshell
model = ifcopenshell.open('sample_floor_wall.ifc')

In [12]:
wall = model.by_type('IfcWall')[0]
print(wall.is_a())

IfcWall


In [15]:
import ifcopenshell.util
import ifcopenshell.util.element
print(ifcopenshell.util.element.get_psets(wall))

{'Pset_WallCommon': {'IsExternal': True, 'ThermalTransmittance': 0.23592605993668123, 'id': 216, 'Reference': 'Wall-Ext_102Bwk-75Ins-100LBlk-12P', 'LoadBearing': False, 'ExtendToStructure': False}, 'Qto_WallBaseQuantities': {'Brick, Common': {'id': 195, 'type': 'IfcPhysicalComplexQuantity', 'Discrimination': 'Layer', 'properties': {'Width': 102.5}}, 'Fiberglass Batt': {'id': 196, 'type': 'IfcPhysicalComplexQuantity', 'Discrimination': 'Layer', 'properties': {'Width': 75.0}}, 'Concrete Masonry Units _Low Density': {'id': 197, 'type': 'IfcPhysicalComplexQuantity', 'Discrimination': 'Layer', 'properties': {'Width': 100.0}}, 'Plaster': {'id': 198, 'type': 'IfcPhysicalComplexQuantity', 'Discrimination': 'Layer', 'properties': {'Width': 12.5}}, 'Width': 290.0, 'Height': 3999.9999999999995, 'Length': 9100.0, 'GrossVolume': 11.213214182027313, 'GrossSideArea': 38.666255800094184, 'NetSideArea': 38.666255800094184, 'NetVolume': 10.997633900337163, 'id': 206}, 'Pset_EnvironmentalImpactIndicators

In [17]:
import ifcopenshell.util.element

wall = model.by_type("IfcWall")[0]
# Walls are typically located on a storey, equipment might be located in spaces, etc
container = ifcopenshell.util.element.get_container(wall)
# The wall is located on Level 01
print(f"The wall is located on {container.Name}")

The wall is located on Level 0


In [ ]:
# import xlsxwriter module
import xlsxwriter
 
# Workbook() takes one, non-optional, argument 
# which is the filename that we want to create.
workbook = xlsxwriter.Workbook('hello.xlsx')
 
# The workbook object is then used to add new 
# worksheet via the add_worksheet() method.
worksheet = workbook.add_worksheet()
 
# Use the worksheet object to write
# data via the write() method.
worksheet.write('A1', 'Hello..')
worksheet.write('B1', 'Geeks')
worksheet.write('C1', 'For')
worksheet.write('D1', 'Geeks')
 
# Finally, close the Excel file
# via the close() method.
workbook.close()




In [1]:
import ifcopenshell
print(ifcopenshell.version)
model = ifcopenshell.file()


v0.7.0-f0e03c79d


No stream support: No module named 'lark'


In [65]:
# # Loading the PDF file
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.pdf") # PDF file path


# # loader = PyPDFLoader(r"C:\Users\angad\OneDrive\Desktop\LangChain\india-national-building-code-nbc-2016-vol-1 (1) (1).pdf")
# pages = loader.load_and_split() # essentially splits the pdf into pages with the `page_content` as the text information and `metadata` as the source


In [66]:
# from langchain.document_loaders import TextLoader
# from langchain.chains import ConversationalRetrievalChain
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import VectorDBQA
# from langchain.chains import RetrievalQA
# # from openai import OpenAI
# # import chromadb

# embeddings = OpenAIEmbeddings() # creating the embeddings
# llm = ChatOpenAI() # the llm model 

# documents = pages

# # concatenating all the text in the pdf into one string
# text = ""
# for i in pages:
#     text += i.page_content

# # text splitter
# text_splitter = CharacterTextSplitter(
#         separator="\n",
#         chunk_size=1500,
#         chunk_overlap=500,
#         length_function=len,
#     )
# chunks = text_splitter.split_text(text)


In [67]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter


loader = TextLoader("fire_life_safety.md")

# loader = TextLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.md")
md_text = loader.load()[0].page_content

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_text)

# Char-level splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size = 1000
# chunk_overlap = 500
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size, chunk_overlap=chunk_overlap
# )

# text splitter
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=2000,
        chunk_overlap=1000,
        length_function=len,
    )

# Split
splits = text_splitter.split_documents(md_header_splits)
splits
chunks = [i.page_content for i in splits]



In [68]:
# import pickle

# with open('vectorstore_bsc', 'rb') as file:
#     # Use pickle.dump() to serialize and save the object to the file
#     vectorstore = pickle.load(file)

In [69]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

# HuggingFace Embeddings
# embeddings = HuggingFaceInstructEmbeddings(
#     query_instruction="Represent the query for retrieval: "
# )
embeddings = OpenAIEmbeddings() # creating the embeddings
llm = ChatOpenAI(model = 'gpt-4') # the llm model 

vectorstore = FAISS.from_texts(chunks, embeddings) 

In [70]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4}) # setting up the text retriever. This allows us to choose the number of documents to be returned

In [71]:
# # GPT4All is a local GPT model, change the model to the location on your PC where the model is installed

# from langchain.llms import GPT4All

# llm = GPT4All(
#     model="/Users/rlm/Desktop/Code/gpt4all/models/nous-hermes-13b.ggmlv3.q4_0.bin", # change this with your path for GPT4ALL
#     max_tokens=2048, # max tokens to generate
# )

In [72]:
# Create retrieval QA chain
# from langchain.memory import ConversationSummaryMemory
# memory = ConversationSummaryMemory(llm=llm)
# from langchain.memory import ConversationBufferWindowMemory




# qa_chain = RetrievalQA.from_chain_type(
#     retriever=retriever,
#     llm=llm,
#     memory=ConversationBufferWindowMemory(k=5), # your preferred LLM 
# )

# Ask a question
# query = "what is the title?"
# result = qa_chain.run(query)
# print(result)

In [73]:
def split_string(input_string, max_length=180):
    # max_length = 125
    chunks = []

    current_chunk = ""
    for char in input_string:
        if char == '\n' or len(current_chunk) == max_length:
            chunks.append(current_chunk)
            current_chunk = ""
        else:
            current_chunk += char

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [74]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT



from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

bot = ConversationalRetrievalChain.from_llm(
    llm, retriever, memory=memory, verbose=False
)

In [75]:
query = """Figure out a few fundamental principles that can encompass all rules. An example of fundamental rule could
 calculating the distance of something from something else. These fundamental principles are such that code for each can be written in pyrevit."""
result = bot({"question": query})#

large_string = result['answer']

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

Based on the provided context, here are a few fundamental principles that can be incorporated into a PyRevit code:

1. Classification of Building: The building can be classified based on its type such as high-rise, educational, institutional, business, mercantile, industrial, storage, hazardous 
nd mixed occupancies. The code can be written to categorize the building based on these classifications.

2. Calculation of Building Dimensions: The code can be written to calculate the floor area ratio (FAR), the width of streets providing approach to the building, and the required op
n space around or inside buildings. This would also include calculations for the area of openings in fire resistant walls and floors.

3. Fire Resistance Rating: The code can be written to calculate the fire resistance ratings of structural and non-structural elements based on their types of construction.

4. Egress Requirements: The code can determine the number and size of exits based on the occupancy and size 

In [76]:
query = "Give rules related to point 3"
result = bot({"question": query})#
result['answer']

"According to the provided context, it is required that a structural and/or non-structural element/component shall have the requisite fire resistance rating as per Table 1. The fire resistance rating for the structural and non-structural elements shall be based on guidelines as per approved and accepted standards. The fire rating shall be validated and certified with a view to meeting the requirements of Table 1.\n\nIn the absence of any validated/certified rating, guidance may be obtained from the information available in Annex C. The fire resistance of a building or its structural and non-structural elements is expressed in minutes against a specified fire load which is expressed in kcal/m^2, and against a certain intensity of fire. The fire-resistance test for structural element shall be done in accordance with accepted standards.\n\nTable 1 provides the fire resistance ratings of structural and non-structural elements in minutes for different types of construction (Type 1, Type 2, 

In [77]:
query = "Which rule encompasses calculation of distance of some point to something else"
result = bot({"question": query})#
result['answer']

"The fundamental principle that includes the calculation of the distance from one point to something else is the determination of the 'Building, Height of' and the 'Travel Distance' to exits in the building. The height of the building is calculated from the average level of the ground around and contiguous to the building to the terrace of the last livable floor. On the other hand, the travel distance is used to determine the accessibility and location of exits within a building, ensuring that the travel distance on the floor shall not exceed the distance given in the code. This principle is particularly important in ensuring safety during emergencies."

In [11]:
p = 0
s = 12*1000
r = 0.148
n = 20
for i in range(n):
    p = p+s
    p = p*(1+r)

print(p)
print(p-s*n)

1378211.9569600543
1138211.9569600543


In [10]:
s*n

6000000

In [15]:
query = "The other rules require what kind of stuff other than counting the total number of certain elements or calculating the distance?"
result = bot({"question": query})#
result['answer']

'The rules mentioned in the context provided also mandate the following requirements apart from counting the total number of certain elements or calculating distance:\n\n1. Rules govern the height for different types of buildings according to their occupancy, the width of streets providing approach to the building, permitted floor area ratio (FAR), and permitted open space around or inside buildings.\n\n2. Adequate passageway and clearances are required for firefighting vehicles.\n\n3. Attention must be paid to factors that limit fire spread when designing openings in fire-resistant walls and floors.\n\n4. A doorway or opening in a fire-resistant wall on any floor should not exceed specified dimensions. They also need to be protected with fire-resisting doors of a certain fire rating.\n\n5. Escalator openings must have provisions to avoid smoke spill and need to have sprinklers installed on all sides of the cut out opening on each floor.\n\n6. Certain systems are required for specific 

In [16]:
query = "List all the fundamental things such as calculating the distance or counting the numeber of elements etc which can encompass all"
result = bot({"question": query})#
result['answer']

'The basic principles common to all rules include:\n\n1. Calculation of distances: This involves measuring the travel distance from the most remote point within a storey or a mezzanine floor along the natural and unobstructed path of horizontal or vertical egress travel to the door to an exit. The travel distance on the floor should not exceed the distance given in the specific table (referenced as Table 5 in the text).\n\n2. Number and arrangement of exits: The minimum required number of exits in a building is determined based on occupant load and width required per person as appropriate to the type of exit for respective occupancies. Exits should be placed as remote from each other as possible and arranged to provide direct access in separate directions from any point in the area served.\n\n3. Capacity of exit access: The width of corridors, aisles or ramps required for exit access should be sufficient to ensure a smooth flow of occupants to the exit. The calculation of capacity of e

In [18]:

large_string = result['answer']

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

The basic principles common to all rules include:

1. Calculation of distances: This involves measuring the travel distance from the most remote point within a storey or a mezzanine floor along the natural and unobstructed path of 
orizontal or vertical egress travel to the door to an exit. The travel distance on the floor should not exceed the distance given in the specific table (referenced as Table 5 in th
 text).

2. Number and arrangement of exits: The minimum required number of exits in a building is determined based on occupant load and width required per person as appropriate to the type
of exit for respective occupancies. Exits should be placed as remote from each other as possible and arranged to provide direct access in separate directions from any point in the 
rea served.

3. Capacity of exit access: The width of corridors, aisles or ramps required for exit access should be sufficient to ensure a smooth flow of occupants to the exit. The calculation 
f capacity of exit acc

In [19]:
query = 'are there any more?'
result = bot({"question": query})#
result['answer']

'The document does not provide any additional fundamental principles that can encompass all rules. However, it does mention that the purpose of this Part of the Code is to minimize danger to life and property from fire through occupancy-wise classification, constructional aspects, egress requirements and protection features. It also emphasizes the importance of maintaining all means of exit and fire protection measures during construction, repairs, alterations or additions to a building.'

In [ ]:
from chromadb.utils import embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name="text-embedding-ada-002"
            )

In [1]:
import sys
print(sys.path)

['c:\\Users\\angad\\OneDrive\\Desktop\\LangChain\\Revit-LLM-assistant', '', 'C:\\Users\\angad\\anaconda3\\envs\\revit_llm_env\\Lib\\site-packages', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\python311.zip', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\DLLs', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\Lib', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\Lib\\site-packages', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\Lib\\site-packages\\win32', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\angad\\anaconda3\\envs\\llm_env\\Lib\\site-packages\\Pythonwin']


In [ ]:
client = chromadb.Client()
collection = client.get_or_create_collection("angad-msc",embedding_function=openai_ef)

In [ ]:
# Adding the text and metadata to chromadb
documents = chunks
metadatas = [{"source":"angad_msc"} for i in range(len(chunks)) ]
ids = [str(i) for i in range(len(chunks))]
collection.add(
            documents = documents,
            metadatas = metadatas,
            ids = ids
    )

In [ ]:
# question = "Who is my supervisor?"
question = "summarise section 3"
results = collection.query(query_texts = [question], n_results = 5)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

results = collection.query(query_texts = [question], n_results = 2)

template = """The following piece of text is given: {text}. Please answer any following questions ONLY using THIS piece of text in a brief manner."""
human_template = "{question}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() 
context = ''
for i in range(len(results['documents'])):
    context = context + results['documents'][0][i] + '\n'
chain.invoke({"text": context, "question": question})


In [ ]:
llm = ChatOpenAI()
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=True
    )